# 빅데이터분석기사 실기 대비

4회차 기출 문제 풀이: 자세한 설명 [링크](https://djccnt15.github.io/dataanalysis/bigdata_certi_05/)  

문제 출처  

- [4th-type1-python](https://www.kaggle.com/code/agileteam/4th-type1-python/notebook)
- [4th-t2-python](https://www.kaggle.com/code/agileteam/4th-t2-python/notebook)

데이터 출처  

- [Big Data Analytics Certification KR 2022](https://www.kaggle.com/competitions/big-data-analytics-certification-kr-2022/data)

## 작업형 1

### 문제 1-1

age 컬럼의 3사분위수와 1사분위수의 차를 절대값으로 구하고, 소수점 버려서, 정수로 출력

In [1]:
import pandas as pd

df = pd.read_csv('data/4th/basic1.csv')

In [2]:
res = int(abs(df['age'].quantile(0.75) - df['age'].quantile(0.25)))

print(res)

50


### 문제 1-2

(loves 반응 + wows 반응) / (reactions 반응) 비율이 0.4보다 크고 0.5보다 작으면서, status_type이 'video'인 데이터의 개수  

In [3]:
import pandas as pd

df = pd.read_csv('data/4th/fb.csv')

In [4]:
ratio = (df['loves'] + df['wows']) / df['reactions']
cond_1 = ratio > 0.4
cond_2 = ratio < 0.5
cond_3 = df['type'] == 'video'
res = len(df.loc[cond_1 & cond_2 & cond_3])

print(res)

90


### 문제 1-3

date_added가 2018년 1월 이면서 country가 United Kingdom 단독 제작인 데이터의 개수  

In [5]:
import pandas as pd

df = pd.read_csv('data/4th/nf.csv')

In [6]:
df['date_added'] = pd.to_datetime(df['date_added'])

In [7]:
cond_1 = df['country'] == "United Kingdom"
cond_2 = df['date_added'].dt.year == 2018
cond_3 = df['date_added'].dt.month == 1
res = len(df.loc[cond_1 & cond_2 & cond_3])

print(res)

6


## 작업형 2

기존 고객 분류 자료를 바탕으로 신규 고객이 어떤 분류에 속할지 예측  

- 예측할 값(y): "Segmentation" (1,2,3,4)

**제출 형식**

```
ID,Segmentation
458989,1
458994,2
459000,3
459003,4
```

### 데이터 읽기

In [8]:
import pandas as pd

In [9]:
train = pd.read_csv('data/4th/train.csv')
test = pd.read_csv('data/4th/test.csv')

### EDA

In [10]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               6665 non-null   int64  
 1   Gender           6665 non-null   object 
 2   Ever_Married     6665 non-null   object 
 3   Age              6665 non-null   int64  
 4   Graduated        6665 non-null   object 
 5   Profession       6665 non-null   object 
 6   Work_Experience  6665 non-null   float64
 7   Spending_Score   6665 non-null   object 
 8   Family_Size      6665 non-null   float64
 9   Var_1            6665 non-null   object 
 10  Segmentation     6665 non-null   int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 572.9+ KB
None


In [11]:
train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,4
1,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,2
2,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,2
3,461319,Male,Yes,56,No,Artist,0.0,Average,2.0,Cat_6,3
4,460156,Male,No,32,Yes,Healthcare,1.0,Low,3.0,Cat_6,3


In [12]:
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2154 entries, 0 to 2153
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               2154 non-null   int64  
 1   Gender           2154 non-null   object 
 2   Ever_Married     2154 non-null   object 
 3   Age              2154 non-null   int64  
 4   Graduated        2154 non-null   object 
 5   Profession       2154 non-null   object 
 6   Work_Experience  2154 non-null   float64
 7   Spending_Score   2154 non-null   object 
 8   Family_Size      2154 non-null   float64
 9   Var_1            2154 non-null   object 
dtypes: float64(2), int64(2), object(6)
memory usage: 168.4+ KB
None


In [13]:
print(train['Segmentation'].unique())

[4 2 3 1]


In [14]:
test.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,458989,Female,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6
1,458994,Male,Yes,37,Yes,Healthcare,8.0,Average,4.0,Cat_6
2,459000,Male,Yes,59,No,Executive,11.0,High,2.0,Cat_6
3,459003,Male,Yes,47,Yes,Doctor,0.0,High,5.0,Cat_4
4,459005,Male,Yes,61,Yes,Doctor,5.0,Low,3.0,Cat_6


### 테이블 병합 꼼수

In [15]:
df = pd.concat([train, test])

cols_obj = df.select_dtypes(include='object').columns
cols_num = [col for col in df.columns if col not in cols_obj and col not in ['ID', 'Segmentation']]

In [16]:
print(cols_obj)

Index(['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score',
       'Var_1'],
      dtype='object')


In [17]:
print(cols_num)

['Age', 'Work_Experience', 'Family_Size']


In [18]:
id = test.iloc[:, 0]

In [19]:
print(id)

0       458989
1       458994
2       459000
3       459003
4       459005
         ...  
2149    467950
2150    467954
2151    467958
2152    467961
2153    467968
Name: ID, Length: 2154, dtype: int64


### Label Encoding

In [20]:
[print(f'{col}={df[col].unique()}') for col in cols_obj]

Gender=['Male' 'Female']
Ever_Married=['No' 'Yes']
Graduated=['No' 'Yes']
Profession=['Healthcare' 'Engineer' 'Lawyer' 'Artist' 'Doctor' 'Homemaker'
 'Entertainment' 'Marketing' 'Executive']
Spending_Score=['Low' 'High' 'Average']
Var_1=['Cat_4' 'Cat_6' 'Cat_7' 'Cat_3' 'Cat_1' 'Cat_2' 'Cat_5']


[None, None, None, None, None, None]

In [21]:
cols_obj  # type: ignore

Index(['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score',
       'Var_1'],
      dtype='object')

In [22]:
cols_binary = ['Gender', 'Ever_Married', 'Graduated']
cols_spend = ['Low', 'Average', 'High']
cods_var = ['Cat_1', 'Cat_2', 'Cat_3', 'Cat_4', 'Cat_5', 'Cat_6', 'Cat_7']

In [23]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
for col in cols_binary:
    df[col] = encoder.fit_transform(df[col])

In [24]:
encoder_spend = LabelEncoder()
encoder_spend.fit(cols_spend)
df['Spending_Score'] = encoder_spend.transform(df['Spending_Score'])

In [25]:
encoder_var = LabelEncoder()
encoder_var.fit(cods_var)
df['Var_1'] = encoder_var.transform(df['Var_1'])

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8819 entries, 0 to 2153
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8819 non-null   int64  
 1   Gender           8819 non-null   int32  
 2   Ever_Married     8819 non-null   int32  
 3   Age              8819 non-null   int64  
 4   Graduated        8819 non-null   int32  
 5   Profession       8819 non-null   object 
 6   Work_Experience  8819 non-null   float64
 7   Spending_Score   8819 non-null   int32  
 8   Family_Size      8819 non-null   float64
 9   Var_1            8819 non-null   int32  
 10  Segmentation     6665 non-null   float64
dtypes: float64(3), int32(5), int64(2), object(1)
memory usage: 654.5+ KB


In [27]:
df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,1,0,22,0,Healthcare,1.0,2,4.0,3,4.0
1,466315,0,1,67,1,Engineer,1.0,2,1.0,5,2.0
2,461735,1,1,67,1,Lawyer,0.0,1,2.0,5,2.0
3,461319,1,1,56,0,Artist,0.0,0,2.0,5,3.0
4,460156,1,0,32,1,Healthcare,1.0,2,3.0,5,3.0


### One-Hot Encoding

In [28]:
df = pd.get_dummies(data=df, columns=['Profession'])

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8819 entries, 0 to 2153
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        8819 non-null   int64  
 1   Gender                    8819 non-null   int32  
 2   Ever_Married              8819 non-null   int32  
 3   Age                       8819 non-null   int64  
 4   Graduated                 8819 non-null   int32  
 5   Work_Experience           8819 non-null   float64
 6   Spending_Score            8819 non-null   int32  
 7   Family_Size               8819 non-null   float64
 8   Var_1                     8819 non-null   int32  
 9   Segmentation              6665 non-null   float64
 10  Profession_Artist         8819 non-null   uint8  
 11  Profession_Doctor         8819 non-null   uint8  
 12  Profession_Engineer       8819 non-null   uint8  
 13  Profession_Entertainment  8819 non-null   uint8  
 14  Professi

### 테이블 분리

In [30]:
train = df.iloc[:train.shape[0]]
test = df.iloc[train.shape[0]:]

In [31]:
test.drop(columns=['Segmentation', 'ID'], inplace=True)

C:\Users\wizcore\AppData\Local\Temp\ipykernel_24208\3233341843.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(columns=['Segmentation', 'ID'], inplace=True)


### 종속변수/독립변수 분리

In [32]:
import numpy as np

endog = np.array(train['Segmentation']).reshape(-1, 1)
exog = train.drop(columns=['Segmentation', 'ID'])

### 모델 생성/학습/평가

In [33]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier as KNN

endog = endog.flatten()

score = [{'i': i, 'score_mean': cross_val_score(KNN(n_neighbors=i), X=exog, y=endog, cv=5).mean()} for i in range(10, 500, 10)]
score = pd.DataFrame(score).sort_values(by='score_mean', ascending=False)

print(score.head())

    i  score_mean
2  30    0.480870
1  20    0.476669
3  40    0.474119
5  60    0.471118
4  50    0.470818


In [34]:
model = KNN(n_neighbors=score.iloc[0, 0])
model.fit(X=exog, y=endog)

print(model.classes_)

[1. 2. 3. 4.]


### 데이터 예측

In [35]:
predict = model.predict(test)

print(predict)

[1. 3. 3. ... 2. 3. 3.]


In [36]:
predict_proba = model.predict_proba(test)

print(predict_proba)

[[0.36666667 0.3        0.13333333 0.2       ]
 [0.23333333 0.3        0.4        0.06666667]
 [0.1        0.36666667 0.53333333 0.        ]
 ...
 [0.26666667 0.43333333 0.06666667 0.23333333]
 [0.16666667 0.33333333 0.43333333 0.06666667]
 [0.26666667 0.2        0.3        0.23333333]]


In [37]:
print(len(id), len(predict))

2154 2154


In [38]:
result = pd.DataFrame({'ID': id, 'Segmentation': predict})
result['Segmentation'] = result['Segmentation'].astype(int)

print(result.head())

       ID  Segmentation
0  458989             1
1  458994             3
2  459000             3
3  459003             2
4  459005             2


In [39]:
result.to_csv('result.csv', index=False)